In [1]:
## Dependencies

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from area import area

# ETL Notebook
## New notebook to continue to streamline ETL of baseball field data

### incorporate the working parts of the kml parse book
* leverage the better organization and tagging of the folders to populate the level field in the resulting dataframe
    * should be able to break fields down into following categories based on folder name along
        * high_school (HS or high school in name)
        * youth (youth)
        * college (college)
        * pro (pro)
        * muni (muni) - for municipally owned or public park fields that don't have an active pro team
        * MLB (mlb) - just 3 parrks for now. comerica, coors, and fenway
        

In [2]:
#### Load data from kml file exported by Google Earth

file_path = ('data/kml/ballparks.kml')

with open(file_path) as file:

    xml_data = file.read()



# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
# layers = soup.Document.Folder.Folder
# polygons = soup.Document.Folder.Placemark.Polygon

In [3]:
## Create a dataframe to hold the data parsed from xml
df = pd.DataFrame(columns=['field', 'foul', 'fop'])


## Loop through the folders and extract the data
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\202165734.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\202165734.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\202165734.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\202165734.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\i

In [4]:

## Cleaning
# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 

## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop any rows with empty fields
df = df[(df != 0).all(1)]

# ## remove any numberic characters and . from field names
# # There shouldn't be any anymore because of the validation I did before exporting kml
# df['field'] = df['field'].str.replace(r'\d+', '')

In [5]:
## Parse field names to get level column using regex
import re
re_mlb = re.compile(r'mlb', re.IGNORECASE)
re_pro = re.compile(r'pro', re.IGNORECASE)
re_college = re.compile(r'college', re.IGNORECASE)
re_youth = re.compile(r'youth', re.IGNORECASE)
re_muni = re.compile(r'muni', re.IGNORECASE)

df['level'] = df['field'].apply(lambda x: 'mlb' 
        if re_mlb.search(x) else 'pro' 
        if re_pro.search(x) else 'college' 
        if re_college.search(x) else 'youth' 
        if re_youth.search(x) else 'muni' 
        if re_muni.search(x) else 'high_school')

# clean up the field names
# remove the level from the field name
df['field'] = df['field'].str.replace(r'MLB', '')
df['field'] = df['field'].str.replace(r'pro', '')
df['field'] = df['field'].str.replace(r'college', '')
df['field'] = df['field'].str.replace(r'High School', 'HS')
# remove - from end of field name
df['field'] = df['field'].str.replace(r'- $', '')

## Output test csv
# df.to_csv('TEMP/level2_tost.csv', index=False)

C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\2064707252.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  df['field'] = df['field'].str.replace(r'- $', '')


In [6]:
## Transform DATA
# taken from clean_notebook_parse

## Create a (lat, long) pair for home plate 
df['home_plate'] = df['foul'].str.split(' ').str[0]

## Apply lambda function to format polygon coordinates and pass to the area function
# #OPutput in square meters
df['foul_area'] = df['foul'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))
df['fop_area'] = df['fop'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))

# Convert the area to square feet
df['foul_ft'] = df['foul_area'].apply(lambda x: x*10.7639)
df['fop_ft'] = df['fop_area'].apply(lambda x: x*10.7639)
# round the area to an integer
df['foul_ft'] = df['foul_ft'].round(0)
df['fop_ft'] = df['fop_ft'].round(0)

# drop the square meter area columns
df = df.drop(['foul_area', 'fop_area'], axis=1)

# drop the polygon coordinates
df = df.drop(['foul', 'fop'], axis=1)

# calculate the ratio of foul ground to total area and convert to percentage
df['foul_pct'] = (df['foul_ft']/(df['fop_ft']+df['foul_ft']))*100
# round the percentage to 2 decimal places
df['foul_pct'] = df['foul_pct'].round(2)

### Split the home_plate coordinates into lat and long columns
df['h_lon'] = df['home_plate'].str.split(',').str[0]
df['h_lat'] = df['home_plate'].str.split(',').str[1]

## Drop the home_plate column
df = df.drop(['home_plate'], axis=1)

# # Outputs a clean CSV with area and home plate coordinates
# df.to_csv('data/viz/area_table.csv', index=False)





## Working Above - can consol

### Adding fuzzy matching blocks to connect to enrollment data, ect

In [7]:

### Read the enrollment table from MHSAA website - 2022 enrollment
mhsaa_df = pd.read_csv('data/school_info/mhsaa_enrolment_2022.csv')

## Take area_df from above
area_df = df


### Import area df from a csv file (used for testing purposes)
# ### Open area data and merge with enrollment data
# area_df = pd.read_csv('data/viz/area_table.csv')

# area_df.head()

# area_df['level'].value_counts()



# ## export the mhsaa_df to csv
# mhsaa_df.to_csv('TEMP/mhsaa_df.csv', index=False)




In [8]:
# select just the high school level
hs_df = area_df[area_df['level'] == 'high_school']
other_df = area_df[area_df['level'] != 'high_school']

hs_df.info()
mhsaa_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 1 to 301
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   field     218 non-null    object 
 1   level     218 non-null    object 
 2   foul_ft   218 non-null    float64
 3   fop_ft    218 non-null    float64
 4   foul_pct  218 non-null    float64
 5   h_lon     218 non-null    object 
 6   h_lat     218 non-null    object 
dtypes: float64(3), object(4)
memory usage: 13.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   school_id    746 non-null    int64 
 1   school_name  746 non-null    object
 2   students     746 non-null    int64 
 3   enrollment   746 non-null    int64 
 4   division     746 non-null    object
dtypes: int64(3), object(2)
memory usage: 29.3+ KB


In [9]:
### DOESNT MATCH VERY WELL

# # Using fuzzywuzzy
# # Since there are no examples with the fuzzywuzzy package, here's a function I wrote which will return all matches based on a threshold you can set as a user:
# # https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas

# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# # Function for fuzzy matching

# def fuzzy_merge(df_1, df_2, key1, key2, threshold=85, limit=1):
#     """
#     :param df_1: the left table to join
#     :param df_2: the right table to join
#     :param key1: key column of the left table
#     :param key2: key column of the right table
#     :param threshold: how close the matches should be to return a match, based on Levenshtein distance
#     :param limit: the amount of matches that will get returned, these are sorted high to low
#     :return: dataframe with boths keys and matches
#     """
#     s = df_2[key2].tolist()
    
#     m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
#     df_1['matches'] = m
    
#     m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
#     df_1['matches'] = m2
    
#     return df_1

In [10]:
# fuszzy = fuzzy_merge(hs_df, mhsaa_df, 'field', 'school_name', 90, 1)

# fuszzy.head()

# ### Output the fuzzy merged data
# fuszzy.to_csv('TEMP/fuzzy_merged4.csv', index=False)

In [11]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

hs_names = hs_df['field'].tolist()
enroll_names = mhsaa_df['school_name'].tolist()
# id_list = mhsaa_df['school_id'].tolist()

# set the treshold for the fuzzy match
treshold = 90
mat1 = []


# output a list of the school_id from the mhsaa_df based on the fuzzy match


## do the fuzzy match
for i in hs_names:
    mat1.append(process.extract(i, enroll_names, limit=1, scorer=fuzz.token_set_ratio))

hs_df['match'] = mat1
# mhsaa_df['match'] = mat1

### Take the first item in the tuple

hs_df['match'] = hs_df['match'].apply(lambda x: x[0][0])


C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\1597876719.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['match'] = mat1
C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\1597876719.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['match'] = hs_df['match'].apply(lambda x: x[0][0])


In [12]:
#### Lookup the hs_df['match'] in the mhsaa_df['school_name'] and return the school_id

hs_df['school_id'] = hs_df['match'].apply(lambda x: mhsaa_df[mhsaa_df['school_name'] == x]['school_id'].values[0])

hs_df.head()

## Output a csv to check the fuzzy match
# hs_df.to_csv('TEMP/fuzzy_merged5.csv', index=False)

C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\78640782.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['school_id'] = hs_df['match'].apply(lambda x: mhsaa_df[mhsaa_df['school_name'] == x]['school_id'].values[0])


,field,level,foul_ft,fop_ft,foul_pct,h_lon,h_lat,match,school_id
1,Adrian Lenawee Christian HS,high_school,23325.0,89207.0,20.73,-84.08374689999999,41.9087669,Adrian HS,2294
2,Adrian Lenawee Christian HS- replot,high_school,23515.0,87492.0,21.18,-84.08374923296931,41.90876073773172,Adrian HS,2294
4,Aldai Stevenson HS,high_school,20786.0,88749.0,18.98,-83.01445630000001,42.5871203,Sterling Heights - Utica Stevenson HS,2666
5,Algonac HS,high_school,30427.0,83362.0,26.74,-82.58239759999999,42.6286202,Algonac HS,1668
6,Allen Park HS,high_school,21933.0,94655.0,18.81,-83.2273711,42.2455509,Allen Park HS,5157


In [13]:
### Add the non high schools back to the dataframe

df = hs_df.append(other_df, ignore_index=True)

## Output a csv to check
# df.to_csv('TEMP/fuzzy_merged6.csv', index=False)

C:\Users\Justin\AppData\Local\Temp\ipykernel_4632\3026546024.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = hs_df.append(other_df, ignore_index=True)


In [14]:
### Output new_df to csv

df.to_csv('data/viz/viz_book_area_with_id.csv', index=False)


### A few small problems - Midland Bullock Creek and Grosse Point South didn't import
### the grand Ledge Youth Fields and Moose lodge got included in the high school fields

# The columns are messed up, multiple columns for the level data, ect
## want to leave field name as it is but use the match column as 'team_name'"geometry workbook.ipynb"
# ## populate 'team name' coulmn with values for non high schools
